In [1]:
# train with SAC, stable baseline3
import stable_baselines3
from stable_baselines3 import SAC, PPO
from stable_baselines3.sac import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from wandb.integration.sb3 import WandbCallback
import wandb
# pip install gym-robotics
import matplotlib.pyplot as plt
from gym_robotics.envs.fetch.reach import MujocoPyFetchReachEnv
from gym.wrappers import TimeLimit
from jesnk_utils.rgb_to_video import RGB2VIDEO
import cv2

/research/jesnk_packages/gym_robotics/envs/fetch/reach.py


In [2]:
load_checkpoint_path = "./checkpoint/FetchReach-SAC-dense-20230701_154341.zip"
#load_checkpoint_path = "./checkpoint/PPO-FetchReach-dense-20230621_004058.zip"
model_name = load_checkpoint_path.split("/")[-1].split(".")[0]
rollout_path = f"./checkpoint_rollout/{model_name}/"
model = SAC.load(load_checkpoint_path)

In [4]:
rgb_to_video = RGB2VIDEO()
env = MujocoPyFetchReachEnv(reward_type='dense')
env.render_mode = 'rgb_array'
#env = Monitor(env_eval, log_dir)
#env = TimeLimit(env, max_episode_steps=100)
env = DummyVecEnv([lambda: env])
#env.render_mode = 'rgb_array'

episode_step = 0
episode_num = 0
replay_step = 300
cumulative_reward = 0
frames = []
obs = env.reset()

success = []

trajectories = []

#### NOT IMPLEMENTED ####

for i in range(1,replay_step+1):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = env.step(action)
    cumulative_reward += rewards[0]

    episode_step += 1
    if dones[0]:
        obs = env.reset()
        success.append(info[0]['is_success'])
        episode_step = 0
        cumulative_reward = 0
        episode_num += 1

print(f'episode {i} done')
success_rate = sum(success)/len(success)
print(f'success rate: {success_rate}')

rgb_to_video.set_frames(frames)
rgb_to_video.set_fps(5)
rgb_to_video.save(path=f'{rollout_path}epi{len(success)}_sucrat{success_rate:.3f}.gif',mode='gif')

frames = []
rgb_to_video.container.clear()


/research/jesnk_packages/gym_robotics/envs/robot_env.py:330: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(


episode 300 done
success rate: 0.8888888888888888
